In [1]:
from llm import LLM, LLM_Stream
from db import DB, Corpus, Doc, Line, Form, Predict, PredictRaw, Lemma, LemmaRaw
from datetime import datetime
from sqlmodel import SQLModel, Field, Relationship, UniqueConstraint, create_engine, Session, select, delete
from time import sleep
from textwrap import dedent, wrap
from pandas import DataFrame as df
import json

In [2]:
llm = LLM()
# llm.complete()
# llm.debug()
db = llm.db
db.del_test()
db.stat()

1: corpus
5: doc
2541: line
28300: form
319: predict
319: predictraw
1323: lemma
1323: lemmaraw


In [3]:
for line in llm.db.get_line():
  try:
    print(llm.lemmatize(line))
  except KeyboardInterrupt:
    print('.. вы остановили расчет')
    break

line: 605 ERR: 1 < 2
line: 1282 ERR: 1 < 2
line: 1296 ERR: 1 < 2
line: 1426 ERR: 1 < 2
line: 1533 ERR: id=19107 form='for.an' num=1 line_id=1533 lemma='for.an' != {'word_form': 'for', 'lemma': 'for', 'translation_en': 'for', 'translation_ru': 'для', 'morph_analysis': 'preposition', 'syntax_analysis': 'marker of the infinitive'}
None


/Users/malex/src/nora_python/.venv/lib/python3.12/site-packages/sqlmodel/orm/session.py:127: SAWarning: Object of type <Lemma> not in session, add operation along 'Form.lemmas' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  return super().execute(


line: 1738 ERR: 1 < 2
line: 1771 ERR: 1 < 2
line: 1852 ERR: 1 < 2
line: 1918 ERR: 1 < 2
line: 1923 ERR: 1 < 2
line: 1960 ERR: 1 < 2
line: 2146 ERR: 1 < 2
line: 2276 ERR: 1 < 2
line: 2294 ERR: 1 < 2
line: 2305 ERR: 1 < 2
line: 2510 ERR: 1 < 2
line: 214 ERR: id=3262 form='Seo' num=0 line_id=214 lemma='se' != {'word_form': 'seo', 'lemma': 'she', 'translation_en': 'she', 'translation_ru': 'она', 'morph_analysis': 'pronoun, feminine, nominative singular, third person', 'syntax_analysis': 'subject'}
None
line: 222 ERR: id=3323 form='Seo' num=0 line_id=222 lemma='se' != {'word_form': 'seo', 'lemma': 'she', 'translation_en': 'she', 'translation_ru': 'она', 'morph_analysis': 'pronoun, feminine, nominative singular, third person', 'syntax_analysis': 'subject'}
None
line: 355 ERR: 2 < 3
line: 417 ERR: id=5700 form='Gehyre' num=0 line_id=417 lemma='gehyran' != {'word_form': 'gehyre', 'lemma': 'hyran', 'translation_en': 'to hear', 'translation_ru': 'слушать', 'morph_analysis': 'pronoun + verb (past

In [3]:
# llm.lemmatize()

(244, 0)

In [4]:
# llm_stream = LLM_Stream()
# llm_stream.stream_complete("Напиши короткое стихотворение")
# # llm_stream.debug()

In [5]:
# line: Line = db.tbl(Line, 1).one()
# line.forms

In [ ]:
# s = db.s
# line = llm.db.get_test_line()
# raw = llm.response.raw
# raw["created_at"]
# db.stat()

In [8]:
# s.exec(delete(PredictRaw))
# s.exec(delete(Lemma))
# s.exec(delete(LemmaRaw))
# s.commit()
# db.stat()

In [17]:
# textwrap.shorten

<function textwrap.shorten(text, width, **kwargs)>

In [ ]:
# print(db.one(Predict, 1).raw.content)

In [4]:
df(db.s.exec(
    select(Lemma.eq, Lemma.lemma, Form.lemma, Form.form)
    .join(Form, Form.id == Lemma.form_id)
    .where(Lemma.predict_id == 1)
).all())

,eq,lemma,lemma_1,form
0,False,that,þes,Ðæs
1,False,day,þegen,ðægn
2,False,Philip,Philippus,Philippus
3,False,not,ne,n
4,False,is,wesan,æs
5,False,not,na,na
6,False,filled,gefullian,gefullod
7,True,on,on,on
8,True,God,God,Gode
9,False,because,forðam,forþan


In [5]:
# db.df(Predict, 3)
db.df(Predict)

/Users/malex/src/nora_python/.venv/lib/python3.12/site-packages/sqlmodel/orm/session.py:66: SAWarning: Object of type <Lemma> not in session, add operation along 'Form.lemmas' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  results = super().execute(


,id,line_id,model,done,no_eq,at,load_duration,prompt_eval_duration,eval_duration,prompt_tokens,completion_tokens,temperature,test
0,1,4,mistral:7b,True,25,2025-01-11 18:50:28.836595,5611709,560000000,63074000000,179,2314,0.0,False
1,2,6,mistral:7b,True,13,2025-01-11 20:40:04.691167,5186958,509000000,39727000000,167,1540,0.0,False
2,3,8,mistral:7b,True,12,2025-01-11 20:40:58.710192,5916667,548000000,31325000000,150,1240,0.0,False
3,4,10,mistral:7b,True,13,2025-01-11 20:42:06.542800,5843083,523000000,28839000000,148,1127,0.0,False
4,5,12,mistral:7b,True,11,2025-01-11 20:44:00.771678,5769041,548000000,41758000000,164,1512,0.0,False
5,6,15,mistral:7b,True,3,2025-01-11 20:44:41.286079,6170833,466000000,10391000000,128,433,0.0,False
6,7,17,mistral:7b,True,20,2025-01-11 20:46:20.464959,6091417,532000000,60709000000,173,2101,0.0,False
7,8,18,mistral:7b,True,19,2025-01-11 20:47:22.055271,7614541,605000000,60951000000,180,2124,0.0,False
8,9,19,mistral:7b,True,9,2025-01-11 20:47:48.843750,5597125,447000000,26322000000,143,960,0.0,False
9,10,24,mistral:7b,True,5,2025-01-11 20:50:01.233456,11399167,470000000,12922000000,130,499,0.0,False


In [ ]:
# for line in db.get_line():
#   print(line)
#   sleep(1)

In [5]:
for line in llm.db.get_line():
  try:
    print(llm.lemmatize(line))
  except KeyboardInterrupt:
    print('.. вы остановили расчет')
    break

line: 1 ERR: line_id=1 num=13 lemma='heo' id=14 form='heo' != {'word_form': 'ðurh', 'lemma': 'þurh', 'translation_en': 'through', 'translation_ru': 'через', 'morph_analysis': 'preposition, dative case of þurh (through)', 'syntax_analysis': 'preposition'}
None


/Users/malex/src/nora_python/.venv/lib/python3.12/site-packages/sqlmodel/orm/session.py:127: SAWarning: Object of type <Lemma> not in session, add operation along 'Form.lemmas' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  return super().execute(


line: 2 ERR: line_id=2 num=24 lemma='Alexandria' id=49 form='Alexandria' != {'word_form': 'ðone', 'lemma': 'dæn', 'translation_en': 'that', 'translation_ru': 'этот', 'morph_analysis': 'DET, SG, ACC', 'syntax_analysis': 'determiner'}
None
line: 3 ERR: line_id=3 num=0 lemma='and' id=50 form='And' != {'word_form': 'he', 'lemma': 'hē', 'translation_en': 'he', 'translation_ru': 'он', 'morph_analysis': '3sg pronominal pronoun', 'syntax_analysis': 'subject'}
None
line: 5 ERR: line_id=5 num=0 lemma='his' id=97 form='His' != {'word_form': 'he', 'lemma': 'hē', 'translation_en': 'he', 'translation_ru': 'он', 'morph_analysis': '3sg pronominal pronoun', 'syntax_analysis': 'subject'}
None
line: 7 ERR: line_id=7 num=10 lemma='on' id=148 form='on' != {'word_form': 'uðwytegunge', 'lemma': 'understanding', 'translation_en': 'understanding', 'translation_ru': 'понимание', 'morph_analysis': 'noun, genitive singular feminine', 'syntax_analysis': "attribute of 'Eugenia'"}
None
line: 9 ERR: line_id=9 num=11 

In [8]:
# for line in llm.db.get_line():
#   # db.one(Line, line.id)
#   print(line)
#   sleep(1)

In [6]:
# DB.db_path
! ls -lh 'db/llm.db'

-rw-r--r--@ 1 malex  staff   2.6M Jan 12 21:26 db/llm.db
